## Job Web-Scraping
Note: This is intented for practice to extract real time job descriptions. Please adhere to the Robots.txt

In [36]:
#If you are using Selenium for the first time, please download the webdriver and note the filepath
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.keys import Keys

import pandas as pd
import numpy as np

#Used to allow the browser to load. 
import time

#used as a import for dateing the CSV
import datetime

#NLP libaries
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import pyLDAvis.gensim
from gensim.models import LdaModel
pyLDAvis.enable_notebook()

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt

In [16]:
path_to_chromedriver = "./chromedriver_win32/chromedriver.exe"

In [17]:
#setup browser window
browser = webdriver.Chrome(executable_path= path_to_chromedriver)

In [18]:
#give it the site
url = "https://www.careeronestop.org/Toolkit/Jobs/find-jobs.aspx?keyword=Data%20Analyst&location=78741&radius=5&sortcolumns=accquisitiondate&sortdirections=DSC"
browser.get("https://www.indeed.com/")
time.sleep(1)

In [19]:
job = browser.find_element(by='id',value='text-input-what')
where = browser.find_element(by='id',value='text-input-where')

#sends the key commands to select all and clear
where.send_keys(Keys.CONTROL,'a')
where.send_keys(Keys.DELETE)

#find data science jobs and submit. 
job.send_keys("data scientist")
where.send_keys("Austin, Texas")
browser.find_element_by_css_selector("button").submit()

In [20]:
#finds the distance by the exact location.
select = Select(browser.find_element_by_id("distance_selector"))
select.select_by_value("0")

In [21]:
#sorts the data by date.
browser.find_element_by_link_text('date').click()

In [22]:
#these catch the title of the job and the description in two seperate lists.
j_desc = []
j_title = []

In [23]:
listing_num = len(browser.find_elements_by_xpath('//*[@data-tn-element="jobTitle"]'))

In [24]:
for job in range(1,listing_num):
    try:
        browser.find_elements_by_class_name('jobtitle')[job].click()
        time.sleep(5)
        j_title.append(browser.find_element_by_id("vjs-jobtitle").text)
        j_desc.append(browser.find_element_by_id("vjs-desc").text)
    except:
        if len(browser.window_handles) > 1:
            window = browser.window_handles[1]
            browser.switch_to.window(window)
            browser.title
            time.sleep(2)
            data = browser.find_element_by_class_name("snip").text
            data = data.replace("\n"," ").replace(" "," ")
            j_title.append(browser.title)
            j_desc.append(data)
            browser.close()
            browser.switch_to.window(browser.window_handles[0])
            time.sleep(np.random.randint(1,5))
        else:
            time.sleep(3)
            browser.switch_to.window(browser.window_handles[0])
            pass

In [25]:
len(j_desc),len(j_title)

(1, 1)

In [26]:
browser.close()

In [27]:
data = pd.DataFrame({'job_title': j_title,'job_desc':j_desc})

In [28]:
data.to_csv("./data/"+str(datetime.datetime.today())[:10]+"job_list.csv")

## Using SpaCy for NLP - this is a Work in Progress
Objective: Setup for topic modeling and use LDA to determine feature importance

In [30]:
parser = spacy.lang.en.English()
nlp = spacy.load('en')

In [31]:
lib = []
for i in j_desc:
    lib.append(i)

In [32]:
parsed = []
for sentence in lib:
    parsed.append(nlp(sentence))

In [33]:
lemma = []

In [34]:
for docs in parsed:
    for token in docs:
        lemma.append(token.lemma_)

In [35]:
lemma[:10]

['the',
 'lead',
 'data',
 'scientist',
 'and',
 'cognitive',
 'software',
 'engineer',
 'will',
 'lead']